In [ ]:
import pandas as pd
import requests 
import re
from bs4 import BeautifulSoup
import numpy as np
import time 
from time import sleep
import csv
import progressbar

In [ ]:
#start page with list of states
start_page = 'http://www.ddugjy.gov.in/xxicompleteplan'

In [ ]:
#get the links to the states
response = requests.get(start_page)
start_page = BeautifulSoup(response.text, 'lxml')
start_page

In [ ]:
states = start_page.find_all('a', href=True)

In [ ]:
#states

In [ ]:
#type(states)

In [ ]:
#states[1]

In [ ]:
#create a list of links to the states
#select only the right links
states_link = []
for state in states: 
    link = state['href']
    states_link.append(link)
states_link

In [ ]:
#len(states_link)

In [ ]:
states_link = [state for state in states_link if '/xxicompleteplan/' in state]
len(states_link)

In [ ]:
#states_link

In [ ]:
#create a list of state codes
states_code = []
for link in states_link: 
    state_code = link.split('/')[4]
    states_code.append(state_code)

In [ ]:
len(states_code)

In [ ]:
#request on each link to state
#get for each state a list of districts
#responses_states = []
districts = []

for link in states_link: 
        response = requests.get(link)
        soup = BeautifulSoup(response.text, 'lxml')
        #responses_states.append(soup)        
        #get the link to each district in each state

        for line in soup.find_all('a', href=True): #page source behind each state page: contains list of districts in the state
            link = line['href']
            districts.append(link)
        
#select the right links
districts = [district for district in districts if 'comblockwisexi' in district]

In [ ]:
districts

In [ ]:
len(districts)

In [ ]:
#request on each link to district
#each district contains blocks
#responses_districts = []
blocks = []
count = 0

bar = progressbar.ProgressBar(max_value=len(districts))

for link in districts: 
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'lxml')
    #responses_districts.append(soup) #save quite some memory :)
    for line in soup.find_all('a', href=True):
        link = line['href']
        blocks.append(link)
        
    count = count + 1
    bar.update(count)

#keep the right ones (total = un/de electrified or already electrified and intensive electrification received)
blocks = [block for block in blocks if 'elect_status=U&elect_status1=D&elect_status2=E' in block]

In [ ]:
len(blocks)

In [ ]:
# split the blocks into pieces of 500 blocks each. 
# to track the progress of the requests on the blocks. 
chunks = [blocks[x:x+500] for x in range(0, len(blocks), 500)]

In [ ]:
# chunks is a list of 12 lists of 500 block links each. 
len(chunks)

In [ ]:
# from block links, extract village links 
# show a progress bar for preogresses on chunks. one step for each chunk 
links_village = []

count = 0 
bar = progressbar.ProgressBar(max_value=len(blocks))

for block in blocks: #5875
    response = requests.get(block)
    soup = BeautifulSoup(response.text, 'lxml')
 
    for line in soup.find_all('a', href=True):
        link = line['href']        
        links_village.append(link)
       
    count = count + 1 #next block
    bar.update(count) #n_block/5875

In [ ]:
links_village = [village for village in links_village if 'elect_status=U&elect_status1=D&elect_status2=E' in village ]

In [ ]:
with open("villages.txt", "w") as f: 
    f.write(str(links_village))

In [ ]:
village_code = []
for village in links_village: 
    v_code = village.split('villcd=')[1]
    v_code = re.findall('^[0-9]+', v_code)
    village_code.extend(v_code)

In [ ]:
len(village_code)

In [ ]:
block_code = []
for village in links_village: 
    b_code = village.split('blkcd=')[1]
    b_code = re.findall('^[0-9]+', b_code)
    block_code.extend(b_code)

In [ ]:
len(block_code)

In [ ]:
district_code = []
for village in links_village: 
    d_code = village.split('edtcd=')[1]
    d_code = re.findall('^[0-9]+', d_code)
    district_code.extend(d_code)

In [ ]:
len(district_code)

In [ ]:
state_code = []
for village in links_village: 
    s_code = village.split('estcd=')[1]
    s_code = re.findall('^[0-9]+', s_code)
    state_code.extend(s_code)

In [ ]:
len(state_code)

In [ ]:
s = [i for i in np.arange(0,287691,2000)]

In [ ]:
s

In [ ]:
# make request on each village

coverage = '\xa0\xa0Coverage Category(Un-Electrified/De-Electrified/Electrified)'
RHH_plan = '\xa0\xa0No.of Rural Household(RHH) Connections to be Released' #rural household connection to be released
BPL_plan =  '\xa0\xa0No.of BPL Connections to be Released'
RHH_comp = '\xa0\xa0Rural Households Connections Completed(BPL+APL)'
BPL_comp = '\xa0\xa0Number of BPL Households Connections Completed'
N_habit = '\xa0\xa0Total No. Habitations\n\t\t\t\t\xa0\n\t\t\t\t'

In [ ]:
coverage_status = []
RHH_plan_list = []
BPL_plan_list = []
RHH_comp_list = []
BPL_comp_list = []
N_habitations = []

In [ ]:
len(links_village)

In [ ]:
count = 0
bar = progressbar.ProgressBar(max_value = len(links_village10))

for village in links_village10: 
    sr = []  #single village response
    response = requests.get(village)
    soup = BeautifulSoup(response.text, 'lxml')
    for line in soup.find_all('td'):
        td = line.text
        sr.append(td)
    
    i_cov = sr.index(coverage)
    i_RHH_p = sr.index(RHH_plan)
    i_BPL_p = sr.index(BPL_plan)
    i_RHH_c = sr.index(RHH_comp)
    i_BPL_c = sr.index(BPL_comp)
    i_habit = sr.index(N_habit)
    
    cov = sr[i_cov+1].strip()
    RHH_p = sr[i_RHH_p+1].strip()
    BPL_p = sr[i_BPL_p+1].strip()
    RHH_c = sr[i_RHH_c+1].strip()
    BPL_c = sr[i_BPL_c+1].strip()
    N_Habit = sr[i_habit+1].strip()
    
    coverage_status.append(cov)
    RHH_plan_list.append(RHH_p)
    BPL_plan_list.append(BPL_p)
    RHH_comp_list.append(RHH_c)
    BPL_comp_list.append(BPL_c)
    N_habitations.append(N_Habit)

    with open('villages_lists10.csv', 'w', newline='') as vl: 
        wr = csv.writer(vl, delimiter=',', quoting = csv.QUOTE_ALL)
        wr.writerow(coverage_status)
        wr.writerow(RHH_plan_list)
        wr.writerow(BPL_plan_list)
        wr.writerow(RHH_comp_list)
        wr.writerow(BPL_comp_list)
        wr.writerow(N_habitations)
    
    count = count + 1
    bar.update(count)
    
    for i in s:
        if count == i:
            time.sleep(60)


In [ ]:
len(coverage_status)

In [ ]:
len(N_habitations)

In [ ]:
len(RHH_plan_list)

In [ ]:
data = pd.DataFrame({'elect_status': coverage_status, 'RHH_plan': RHH_plan_list, 'BPL_plan': BPL_plan_list, 'RHH_comp': RHH_comp_list, 'BPL_comp': BPL_comp_list, 'N_habit': N_habitations})

In [ ]:
data

In [ ]:
data.to_csv(r'C:\Users\Mari\Documents\Uni\Python_materials\all_villages.csv')

In [ ]:
codes = pd.DataFrame({'state': state_code, 'district': district_code, 'block': block_code, 'village': village_code})

In [ ]:
codes

In [ ]:
villages_clean = pd.read_csv(r'C:\Users\Mari\Documents\Uni\Python_materials\all_villages_clean.csv')

In [ ]:
villages_clean

In [ ]:
full = pd.concat([codes, villages_clean], axis=1, sort=False)

In [ ]:
full

In [ ]:
full.to_excel(r'C:\Users\Mari\Documents\Uni\Python_materials\villages_clean_full.xlsx')